In [10]:
A quick tutoial using agnvar. We will be using the agnsed mode (AGNsed_var) 

SyntaxError: invalid syntax (<ipython-input-10-80723322931f>, line 1)

In [9]:
#Generic imports
import numpy as np
import matplotlib.pyplot as plt

#Adding src directory to python path - needed as agnvar uses pyNTHCOMP, this is easiest way of NOT breaking code...
import os
import sys
sys.path.append(os.path.abspath('src'))

#Doing example with AGNsed_var (as my preffered model) - however the syntex is same for the others too
from agnvar import AGNsed_var